# Solving the magic square
A magic square is an __n x n__ square where the sum of each row, column and diagonal is equal to the sum of each other
row, column and diagonal.<br>
For this assignment specifically, the assumption is that the magic square is __3 x 3__, and decimal places are 
allowed. All answers must be greater than 0<br>

$M \cdot \vec{r} = \vec{i}$  
$\vec{r} = M^{+}\cdot\vec{i}$  
Additionally, some discrepancies exist between the actual answer and the answer generated by the script, this is due to
floating point inaccuracy.<br>
## The chosen square
The square that will be solved, first in writing and second via a script, goes as follows: <br>
$\begin{array} {|r|r|}
\hline 5 &  &    \\
\hline   &  & 4  \\
\hline   &  & 6  \\
\hline
\end{array}$ <br>
In the code, it is represented like this: (Empty spaces are replaced with 0s) <br>

In [1]:
import numpy as np
magic_square = np.array([
        [5, 0, 0],
        [0, 0, 4],
        [0, 0, 6]])
print(magic_square)

[[5 0 0]
 [0 0 4]
 [0 0 6]]


## System of equations
\begin{array} {|r|r|}
\hline x_1 & x_2 & x_3 \\
\hline x_4 & x_5 & x_6 \\
\hline x_7 & x_8 & x_9 \\
\hline
\end{array} <br>
A magic square can be turned into a system of equations; the sum of each row, column and diagonal must equal $t$<br>
Columns:
> $x_1 + x_2 + x_3 = t$  
$x_4 + x_5 + x_6 = t$  
$x_7 + x_8 + x_9 = t$  

Rows:
>$x_1 + x_4 + x_7 = t$  
$x_2 + x_5 + x_8 = t$  
$x_3 + x_6 + x_9 = t$  

Diagonals:
>$x_1 + x_5 + x_9 = t$  
$x_3 + x_5 + x_7 = t$  

And finally, a special case:  
>$3x_5 = t$ <br>

The rows, columns and diagonals can be separated through the following function:

In [3]:
def get_variants(matrix):
    """Gets columns, rows and diagonals from a 3x3 matrix"""
    row_1 = matrix[0]
    row_2 = matrix[1]
    row_3 = matrix[2]

    column_1 = matrix[:, 0]
    column_2 = matrix[:, 1]
    column_3 = matrix[:, 2]

    diagonal_1 = matrix.diagonal()
    diagonal_2 = np.fliplr(matrix).diagonal()

    rule_indices = [row_1, row_2, row_3, column_1, column_2, column_3, diagonal_1, diagonal_2]

    return rule_indices

For the purposes of this magic square, we know what the values of $x_{1}, x_{6} \space \And \space x_{9} $. Because of
this, we can now turn this into the next set of equations. <br>

Columns:  
>$x_2 + x_3 - t = -x_{1}$  
$x_4 + x_5  - t = -x_6$  
$x_7 + x_8 - t = -x_9$  

Rows:
>$x_4 + x_7 - t = -x_1$  
$x_2 + x_5 + x_8 - t = 0$  
$x_3 - t = - x_6 - x_9$  

Diagonal:
>$x_5 - t = - x_9 - x_1$  
$x_3 + x_5 + x_7 - t = 0$  

Special case:
>$3x_5 - t = 0$  

Finally, this system of equations can be transformed into matrix <br>
$\begin{bmatrix}0  &1  &1  &0  &0  &0  &0  &0  &0 &-1\\
    0  &0  &0  &1  &1  &0  &0  &0  &0 &-1\\
    0  &0  &0  &0  &0  &0  &1  &1  &0 &-1\\
    0  &0  &0  &1  &0  &0  &1  &0  &0 &-1\\
    0  &1  &0  &0  &1  &0  &0  &1  &0 &-1\\
    0  &0  &1  &0  &0  &0  &0  &0  &0 &-1\\
    0  &0  &0  &0  &1  &0  &0  &0  &0 &-1\\
    0  &0  &1  &0  &1  &0  &1  &0  &0 &-1\\
    0  &0  &0  &0  &3  &0  &0  &0  &0 &-1\\ \end{bmatrix}$ and a vector $\begin{bmatrix} 
    -5\\ -4\\ -6\\ -5\\ 0\\ -10\\ -11\\ 0\\ 0 \end{bmatrix}$<br>
This matrix also gets generated by the following python script. (the numbers right of the '=' are added to the vector)

In [7]:
def generate_matrix(magic_square):
    index_list = list()
    for i in range(len(magic_square) * len(magic_square)):
        index_list.append(i)

    index_matrix = np.array(index_list).reshape(magic_square.shape)

    flat_square = magic_square.flatten()

    vector = list()

    rule_indices = get_variants(index_matrix)

    matrix = np.zeros((9, 10), dtype=int)
    # Set the final value in the final row to -1 (the final row needs to be set manually
    matrix[8][9] = -1
    for i in range(len(rule_indices)):
        # Append the total sum of each rule to vector (- row, column, diagonal)
        vector.append(np.sum(flat_square[rule_indices[i]]) * -1)
        # Set the final value in the i-th row to -1
        matrix[i][9] = -1
        # For each index in the rule:
        for index in rule_indices[i]:
            # If the value of that index in flat square is smaller than 1, set the matrix at those indices to 1
            if flat_square[index] < 1:
                matrix[i][index] = 1
    
    # Depending on the value of the middle point, append -3 times that value to the vector
    if flat_square[4] > 0:
        vector.append(flat_square[4] * -3)
    else:
        # Set the value of the final row 4th value to 3
        matrix[8][4] = 3
        # Append 0 to the vector
        vector.append(0)
    return matrix, vector

matrix, vector = generate_matrix(magic_square)
# Matrix
print("Matrix:\n", matrix)
# Vector
print("\nVector:",vector)


Matrix:
 [[ 0  1  1  0  0  0  0  0  0 -1]
 [ 0  0  0  1  1  0  0  0  0 -1]
 [ 0  0  0  0  0  0  1  1  0 -1]
 [ 0  0  0  1  0  0  1  0  0 -1]
 [ 0  1  0  0  1  0  0  1  0 -1]
 [ 0  0  1  0  0  0  0  0  0 -1]
 [ 0  0  0  0  1  0  0  0  0 -1]
 [ 0  0  1  0  1  0  1  0  0 -1]
 [ 0  0  0  0  3  0  0  0  0 -1]]

Vector: [-5, -4, -6, -5, 0, -10, -11, 0, 0]


## Matrix to pseudo inverse
The next step is to get transform the matrix to the shape of __n x n__. This is accomplished by removing one column 
of zeroes. The result of this is, a __9 x 9__:  
$\begin{bmatrix}1  &1  &0  &0  &0  &0  &0  &0 &-1\\
    0  &0  &1  &1  &0  &0  &0  &0 &-1\\
    0  &0  &0  &0  &0  &1  &1  &0 &-1\\
    0  &0  &1  &0  &0  &1  &0  &0 &-1\\
    1  &0  &0  &1  &0  &0  &1  &0 &-1\\
    0  &1  &0  &0  &0  &0  &0  &0 &-1\\
    0  &0  &0  &1  &0  &0  &0  &0 &-1\\
    0  &1  &0  &1  &0  &1  &0  &0 &-1\\
    0  &0  &0  &3  &0  &0  &0  &0 &-1\\ \end{bmatrix}$ <br>
The script: <br>

In [9]:
# Removes one column of zeros 
for column_number in range(matrix.shape[1]):
    if np.sum(matrix[:, column_number]) == 0:
        square_matrix = np.delete(matrix, [column_number], 1)
        break
#Newly formed matrix
print("Newly formed matrix: \n",square_matrix)

Newly formed matrix: 
 [[ 1  1  0  0  0  0  0  0 -1]
 [ 0  0  1  1  0  0  0  0 -1]
 [ 0  0  0  0  0  1  1  0 -1]
 [ 0  0  1  0  0  1  0  0 -1]
 [ 1  0  0  1  0  0  1  0 -1]
 [ 0  1  0  0  0  0  0  0 -1]
 [ 0  0  0  1  0  0  0  0 -1]
 [ 0  1  0  1  0  1  0  0 -1]
 [ 0  0  0  3  0  0  0  0 -1]]


### Arguing for the use of pseudo inverse
Creating a square matrix is _not_ necessary when using pseudo inverse. Only the use of true inverse requires a 
square matrix. The determinant of the matrix resulting from the magic square is 0, 
meaning an inverse of this matrix does not exist. An approximation
of one can be calculated using the pseudo inverse. Because an inverse is necessary for this method, the pseudo inverse 
will be used.
## Calculating pseudo inverse of matrix
Next up: converting the matrix to its pseudo inverse. This is accomplished via the pseudo inverse function from 
numpy.linalg. The following is also the result from the script:
$\begin{bmatrix}0.6666666666666665&-0.33333333333333276&-0.33333333333333354&0.3333333333333329&0.33333333333333354&-0.6666666666666663&-4.768313858119252e-16&1.9239182338055723e-16&-3.942520632758679e-16&\\
-0.11111111111111105&0.4444444444444435&-0.11111111111111135&-0.44444444444444375&0.11111111111111113&0.555555555555556&-0.944444444444444&0.5555555555555552&-0.055555555555555386&\\
-0.22222222222222235&0.8888888888888885&-0.22222222222222235&0.11111111111111181&0.2222222222222222&0.11111111111111167&-0.8888888888888884&0.11111111111111097&-0.1111111111111111&\\
-0.11111111111111119&0.11111111111111104&-0.11111111111111126&-0.11111111111111101&0.11111111111111119&-0.11111111111111087&-0.27777777777777757&0.222222222222222&0.2777777777777779&\\
7.583350576213887e-17&-1.4788773221913045e-16&3.885633397356944e-17&1.4984837407048232e-16&-1.6696891151908122e-16&-1.6077355034824378e-17&5.161200551654006e-16&-2.444293874673816e-16&-9.968083556770366e-18&\\
0.11111111111111135&-0.44444444444444436&0.11111111111111126&0.44444444444444425&-0.1111111111111113&-0.5555555555555549&-0.05555555555555541&0.4444444444444445&0.05555555555555519&\\
-0.5555555555555558&0.5555555555555548&0.44444444444444436&-0.5555555555555548&0.5555555555555555&0.444444444444445&-0.8888888888888882&0.11111111111111038&-0.11111111111111067&\\
0.0&0.0&0.0&0.0&0.0&0.0&0.0&0.0&0.0&\\
-0.2222222222222222&0.2222222222222216&-0.22222222222222257&-0.22222222222222154&0.22222222222222215&-0.2222222222222217&-1.0555555555555554&0.444444444444444&0.05555555555555559&\\ 
\end{bmatrix}$  
Script:

In [10]:
# Pseudo inverse
pseudo_inverse = np.linalg.pinv(square_matrix)
print(pseudo_inverse)

[[ 6.66666667e-01 -3.33333333e-01 -3.33333333e-01  3.33333333e-01
   3.33333333e-01 -6.66666667e-01 -4.76831386e-16  1.92391823e-16
  -3.94252063e-16]
 [-1.11111111e-01  4.44444444e-01 -1.11111111e-01 -4.44444444e-01
   1.11111111e-01  5.55555556e-01 -9.44444444e-01  5.55555556e-01
  -5.55555556e-02]
 [-2.22222222e-01  8.88888889e-01 -2.22222222e-01  1.11111111e-01
   2.22222222e-01  1.11111111e-01 -8.88888889e-01  1.11111111e-01
  -1.11111111e-01]
 [-1.11111111e-01  1.11111111e-01 -1.11111111e-01 -1.11111111e-01
   1.11111111e-01 -1.11111111e-01 -2.77777778e-01  2.22222222e-01
   2.77777778e-01]
 [ 7.58335058e-17 -1.47887732e-16  3.88563340e-17  1.49848374e-16
  -1.66968912e-16 -1.60773550e-17  5.16120055e-16 -2.44429387e-16
  -9.96808356e-18]
 [ 1.11111111e-01 -4.44444444e-01  1.11111111e-01  4.44444444e-01
  -1.11111111e-01 -5.55555556e-01 -5.55555556e-02  4.44444444e-01
   5.55555556e-02]
 [-5.55555556e-01  5.55555556e-01  4.44444444e-01 -5.55555556e-01
   5.55555556e-01  4.4444444

Next, we calculate the matrix-vector product of the pseudo inverse and the vector. The last value in the vector is the 
 total sum, this last value gets removed. We add a '0' to the resulting vector at the index of the column we deleted to 
 retain shape.  
$\vec{o} = \begin{bmatrix} 0\\5.0\\6.5\\7.0\\5.5\\-0.0\\4.5\\6.0\\0.0\\ \end{bmatrix}, \sum_{total} = 16.5$  
Script:

In [11]:
def matrix_vector_product(activation, weights):
    # Matrix vector product from Neural Network assignment
    output_layer = list()
    for w in weights:
        vector = 0.0
        for a in range(len(activation)):
            vector1 = float(float(activation[a]) * float(w[a]))
            vector += vector1
        output_layer.append(vector)
    return output_layer
output_vector = matrix_vector_product(vector, pseudo_inverse)

magic_sum = output_vector.pop()
output_vector.insert(column_number, 0)

print("Output vector:\n", output_vector)
print("\nMagic sum:", magic_sum)

Output vector:
 [0, 5.000000000000003, 6.499999999999991, 6.999999999999988, 5.4999999999999964, -6.286543530599164e-15, 4.49999999999999, 5.999999999999988, 0.0]

Magic sum: 16.499999999999993


## End result:
$\begin{array} {|r|r|}
\hline 5 & 5 & 6.5   \\
\hline 7  & 5.5 & 4  \\
\hline  4.5 & 6 & 6  \\
\hline
\end{array}$ <br>

Script:

In [13]:
flat_nonzeroes = list(magic_square.flatten().nonzero())[0]
solved_vector = output_vector
for given_index in flat_nonzeroes:
    solved_vector[given_index] = magic_square.flatten()[given_index]
solved_square = np.reshape(solved_vector, magic_square.shape)

print(solved_square, magic_sum)


[[5.  5.  6.5]
 [7.  5.5 4. ]
 [4.5 6.  6. ]] 16.499999999999993


#### Extra: 
Going through all columns, rows and diagonals to check if sum equals magic sum with an accuracy of 1 decimal


In [14]:

#     Checking
rules = get_variants(solved_square)
for rule in rules:
    print(rule, '=', np.sum(rule))
    if round(np.sum(rule), 2) == round(magic_sum,2):
        print("True")

[5.  5.  6.5] = 16.499999999999993
True
[7.  5.5 4. ] = 16.499999999999986
True
[4.5 6.  6. ] = 16.49999999999998
True
[5.  7.  4.5] = 16.49999999999998
True
[5.  5.5 6. ] = 16.499999999999986
True
[6.5 4.  6. ] = 16.499999999999993
True
[5.  5.5 6. ] = 16.499999999999996
True
[6.5 5.5 4.5] = 16.49999999999998
True
